# Open-source LLM's as Agents with `ChatHuggingFace`


Open source LLMs are becoming viable general purpose agents. The goal of this notebook is to demonstrate how to make use of open-source LLMs as chat models using [Hugging Face Inference Endpoints](https://huggingface.co/docs/inference-endpoints/index) with [LangChain's `ChatHuggingFace`]() to enable their usage and experimentation with agent-based workflows.

In particular, we will:
1. Utilize the [HuggingFaceEndpoint](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_endpoint.py) (or [HuggingFaceTextGenInference](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_text_gen_inference.py) or [HuggingFaceHub](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_hub.py)) integration to call a [HF Inference Endpoint](https://huggingface.co/inference-endpoints) that's serving an LLM via [Text Generation Inference (TGI)](https://huggingface.co/docs/text-generation-inference/index)
2. Utilize the `ChatHuggingFace` class that interfaces between LangChain's [Chat Messages](https://python.langchain.com/docs/modules/model_io/chat/#messages) and the hosted LLM by leveraging [Hugging Face's Chat Templates](https://huggingface.co/docs/transformers/chat_templating) to power a `ChatAgent` pipeline.
4. Demonstrate how to use an open-source LLM in a zero-shot ReAct Agent workflow, along with an open-source [Prometheus](https://huggingface.co/papers/2310.08491) model to perform "LLM as a Judge"-style evaluations on that Agent's ouputs.
5. Understand how several different open-source LLM's perform as general purpose agents by running an asynchronous evaluation pipeline using Prometheus as the judge. 



> Note: To run this notebook, you'll need to have:
> - an LLM deployed via a Hugging Face Inference Endpoint (the LLM must have a `chat_template` defined in its `tokenizer_config.json`)
> - A Hugging Face Token with access to the deployed endpoint saved as an environment variable: `HUGGINGFACEHUB_API_TOKEN`
> - A SerpAPI key saved as an environment variable: `SERPAPI_API_KEY`


## Setup

In [ ]:
!pip install -q --upgrade transformers langchain text-generation python-dotenv jinja2 langchainhub numexpr datasets tqdm openai sentencepiece protobuf matplotlib google-search-results

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# ruff: noqa: E402

from dotenv import load_dotenv
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
import plotly.express as px
from datasets import load_dataset, concatenate_datasets


from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.agents import load_tools
from langchain.tools.render import render_text_description_and_args
from langchain.chat_models import ChatOpenAI

from prompts import SYSTEM_PROMPT, HUMAN_PROMPT, EVALUATION_PROMPT_TEMPLATE
from evaluation import build_evaluator, evaluate_answers
from run_agents import run_full_tests
from agents import build_hf_agent, build_openai_agent

load_dotenv(override=True)

## 1. Instantiate an LLM

You'll need to have a running Inference Endpoint available.

#### `HuggingFaceHub`

In [ ]:
from langchain.llms.huggingface_hub import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 50,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

#### `HuggingFaceEndpoint`

In [ ]:
from langchain.llms import HuggingFaceEndpoint

endpoint_url = (
    "https://j02macig8fbovqhr.us-east-1.aws.endpoints.huggingface.cloud"  # zephyr
)
endpoint_url = (
    "https://iw8z8uxlp03gvuxc.us-east-1.aws.endpoints.huggingface.cloud"  # mixtral
)
llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 488,
        "top_k": 50,
        "repetition_penalty": 1.03,
    },
)

## 2. Create a wrapper for `BaseChatModel` to apply chat templates

In [ ]:
from langchain.schema import HumanMessage
from langchain_community.chat_models.huggingface import ChatHuggingFace

Instantiate the model and some messages to pass.

In [ ]:
messages = [
    HumanMessage(
        content="You're a helpful assistant. What happens when an unstoppable force meets an immovable object?"
    ),
]

chat_model = ChatHuggingFace(llm=llm)
chat_model.model_id

Call the model.

In [ ]:
res = chat_model.invoke(messages)
print(res.content)

## Tests

Here we'll test out our model as a zero-shot ReAct Agent. The example below is taken from [here](https://python.langchain.com/docs/modules/agents/agent_types/react#using-chat-models).

Configure the agent with a `react-json` style prompt and access to a search engine and calculator.

### Define tools

In [ ]:
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm)
# Rename tools in the same format used by other tools
TOOLS[0].name = "search"
TOOLS[1].name = "calculator"

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(
            SYSTEM_PROMPT + "\nSo, here is my question:" + HUMAN_PROMPT
        ),
    ]
)
prompt = prompt.partial(
    tool_description=render_text_description_and_args(TOOLS),
    tool_names=", ".join([t.name for t in TOOLS]),
)

# define the agent
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

# instantiate AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=TOOLS,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=5,
)

In [ ]:
example = {
    "input": "What is the age of Leonardo DiCaprio's current girlfriend, raised to the power 0.43?"
}

out = agent_executor.invoke(example)

# Evaluation Experiment 

- Find a group of test questions that use a certain set of tools to solve (HotpotQA)
- Run several different models as agent to solve the test questions (OS and proprietary)
- Use a LLM as a judge
- Use OS models as a judge
- Report correlations

### Evaluation dataset

In [ ]:
full_eval_dataset = load_dataset("A-Roucher/agents_small_benchmark", split="train")

# Run tests

In [ ]:
import json

out = None
with open("output/Mixtral-7x8b.json", "r") as f:
    out = json.load(f)
    out = [el for el in out if el["agent_error"] is None]

print(out)
with open("output/Mixtral-7x8b.json", "w") as f:
    json.dump(out, f)

In [ ]:
agent_endpoints = {
    # "Zephyr-7b-beta": "https://ytjpei7t003tedav.us-east-1.aws.endpoints.huggingface.cloud",
    "Mixtral-7x8b": "https://nqoa2is3qe7y82ww.us-east-1.aws.endpoints.huggingface.cloud",
    # 'Yi-34B-Chat': 'https://wtgjpwu76xh7cv7t.us-east-1.aws.endpoints.huggingface.cloud',
    # 'Llama2-7b': 'https://pbg28xzbho42zp1t.us-east-1.aws.endpoints.huggingface.cloud',
    # "SOLAR-10.7B": "https://dxsuz0i09l5zzjh1.us-east-1.aws.endpoints.huggingface.cloud"
    # "openhermes-2-5-mistral-7b": "https://eyq6eaylkve7l1pf.us-east-1.aws.endpoints.huggingface.cloud"
}

agents = {name: build_hf_agent(endpoint) for name, endpoint in agent_endpoints.items()}

# uncomment below to test GPT4 as an agent
# agents["GPT4"] = build_openai_agent(model_id="gpt-4-1106-preview")
# agents["GPT3.5"] = build_openai_agent(model_id="gpt-3.5-turbo-1106")

# run eval
await run_full_tests(dataset=full_eval_dataset, agents=agents)
print("Question answering is complete!")

# Evaluate with LLM-as-a-judge

In [ ]:
# eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
# eval_model_name = "GPT4"

prometheus_endpoint = (
    "https://hg1fvppohh20ufdf.us-east-1.aws.endpoints.huggingface.cloud"
)
eval_chat_model = build_evaluator(prometheus_endpoint)
eval_model_name = "Prometheus-13B-v1.0"

In [ ]:
eval_chat_model.invoke("Hello!")

In [ ]:
for file in tqdm(glob.glob("output/*.json")):
    evaluate_answers(
        file,
        eval_chat_model,
        eval_model_name,
        EVALUATION_PROMPT_TEMPLATE,
    )
print("Evaluation is complete!")

## Visualize results

In [ ]:
result_df = pd.concat([pd.read_json(f) for f in glob.glob("output/*.json")])
try:
    result_df["tools_used"] = result_df["intermediate_steps"].apply(
        lambda row: ([step["tool"] for step in row] if row is not None else None)
    )
    result_df["number_of_distinct_tools_used"] = result_df["tools_used"].apply(
        lambda row: (len(list(set(row))) if row is not None else None)
    )
    result_df["number_of_steps"] = result_df["tools_used"].apply(
        lambda row: (len(row) if row is not None else None)
    )
except:
    pass
result_df["no_prediction"] = result_df["prediction"].apply(
    lambda x: True if x is None else False
)
result_df.head(5)

In [ ]:
def interpret_result(x):
    try:
        return int(x) - 1
    except:
        return 0


result_df["eval_score_Prometheus-13B-v1.0"] = result_df[
    "eval_score_Prometheus-13B-v1.0"
].apply(interpret_result)
result_df["eval_score_GPT4"] = result_df["eval_score_GPT4"].apply(interpret_result)

result_df.groupby("agent_name").agg(
    {
        "eval_score_Prometheus-13B-v1.0": "mean",
        "eval_score_GPT4": "mean",
        "parsing_error": "sum",
        "iteration_limit_exceeded": "sum",
        "no_prediction": "sum",
    }
)

In [ ]:
task_order = {
    "GAIA": 1,
    "GSM8K": 2,
    "HotpotQA-easy": 3,
    "HotpotQA-medium": 4,
    "HotpotQA-hard": 5,
}
result_df = (
    result_df.assign(x=result_df["task"].map(task_order))
    .sort_values("x")
    .drop("x", axis=1)
)
agg_df = result_df.groupby(["agent_name", "task"], sort=False)[
    ["eval_score_Prometheus-13B-v1.0", "eval_score_GPT4"]
].mean()
display(agg_df.unstack())

In [ ]:
agg_df = agg_df / 4 * 100  # set results in percentage

In [ ]:
table_results = agg_df.reset_index().melt(id_vars=["task", "agent_name"])
table_results.columns = ["task", "agent_name", "evaluator", "score"]
table_results["evaluator"] = table_results["evaluator"].str.replace("eval_score_", "")
display(table_results.head())

In [ ]:
result_df["agent_name"].value_counts()

In [ ]:
fig = px.bar(
    table_results,
    x="agent_name",
    y="score",
    color="task",
    facet_row="evaluator",
    title=f"Average Evaluation Score",
    labels={
        "agent_name": "Agent",
        "task": "Task",
        "evaluator": "Evaluator",
        "score": "Performance",
    },
)
fig.update_layout(width=1000, height=600, barmode="group", yaxis_range=[0, 100])
fig.update_traces(texttemplate="%{y:.0f}", textposition="inside")
fig.layout.yaxis.ticksuffix = "%"
fig.show()

### Study intermediate steps

In [ ]:
result_df["has_answer"] = result_df["prediction"].apply(lambda row: (row is not None))
result_df_answers_only = result_df.loc[result_df["has_answer"] == True]
result_df_answers_only["correct_answer"] = (
    result_df_answers_only[f"eval_score_{eval_model_name}"] >= 3
)
aggregated_resuts = result_df_answers_only.groupby(
    ["agent_name", "task", "correct_answer"]
).agg({"number_of_steps": "mean"})

In [ ]:
px.bar(
    aggregated_resuts.reset_index(),
    x="task",
    y="number_of_steps",
    facet_col="agent_name",
    color="correct_answer",
    title="Mean Number of Steps to Solve a Task",
    barmode="group",
    labels={
        "agent_name": "Agent",
        "number_of_steps": "Mean Number of Steps",
        "correct_answer": "Answer correctness",
        "task": "Task",
    },
    width=1500,
)

#### visualizations to make
- avg model score by question difficulty
- count of parsing errors and iteration limit errors

## To-do:
- Generalize evaluation code
- Make evals fully async
- Select 3-5 models to evaluate on
- Run eval
- Write blog post
- Get LangChain PR merged

## Notes

- One of the main challenges with open LLMs is ensuring they adhere to the proper markdown JSON output!
- Another is when they serve as the evaluator, they struggle to handle the default "labeled_criteria" prompt and end up hallucinating. Needs to be modified / simplified.
- I also found that GPT4 actually did pretty poor at judging correctness given the default LangChain prompt!